# <font color="#F48E16">Formación en XAI de Deep Learning: Explicabilidad Genérica</font>

Material generado por <a href="https://www.linkedin.com/in/christian-oliva-moya-ingeniero/">Christian Oliva</a>. Cualquier duda, sugerencia o errata, no duden en contactar.

**Versión 1.0** - 29 de agosto de 2025

In [ ]:
import tensorflow as tf

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import kagglehub
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from tqdm import tqdm

## <font color="#F48E16">Datos</font>

En este notebook se muestra la implementación manual de los diferentes algoritmos de explicabilidad genérica vistos durante el curso, que son los siguientes:

- Importancia por permutación

- Relevancia por oclusión

- SHAP

- LIME

Para ello, se van a utilizar diferentes modelos sencillos de SKLearn sobre un dataset de riesgo financiero para la aprobación de préstamos: **Loan Approval Classification Dataset**

https://www.kaggle.com/datasets/taweilo/loan-approval-classification-data

<hr>

En el notebook se desarrolla el código por completo según una fase sencilla de preprocesamiento de los datos, el entrenamiento de algunos modelos y la explicabilidad utilizando los diferentes algoritmos.

### Descarga de datos de Kaggle

In [ ]:
# Download latest version
path = kagglehub.dataset_download("taweilo/loan-approval-classification-data")

print("Path to dataset files:", path)

### Primer vistazo de los datos

In [ ]:
data = pd.read_csv(path + "/loan_data.csv")
data

In [ ]:
data.info()

In [ ]:
data.isna().sum()

### Preprocesamiento

#### Person Age

In [ ]:
variable = data["person_age"]
variable.hist(bins=31)

In [ ]:
variable = np.clip(variable, 0, 70)
variable = np.log(variable)
variable.hist(bins=31)

#### Person Income

In [ ]:
variable = data["person_income"]
variable.hist(bins=31)

In [ ]:
np.log(variable).hist(bins=31)

#### Person Emp Exp (años de experiencia)

In [ ]:
variable = data["person_emp_exp"]
variable.hist(bins=31)

In [ ]:
variable = np.log(variable+1) # Sumo 1 para evitar -inf
variable.hist(bins=31)

#### Person Home Ownership (situación de propiedad de la vivienda)

Los únicos valores posibles son RENT (alquiler), OWN (propiedad), MORTGAGE (hipoteca) y OTHER (otro).

In [ ]:
variable = data["person_home_ownership"]
variable.unique()

In [ ]:
pd.get_dummies(variable, prefix="person_home_ownership_")

#### Loan Amnt (valor del préstamo)

In [ ]:
variable = data["loan_amnt"]
variable.hist(bins=31)

In [ ]:
variable = np.log(variable)
variable.hist(bins=31)

#### Loan Intent (finalidad del préstamo)

Tiene un conjunto finito de valores: PERSONAL (personal), EDUCATION (educación), VENTURE (médico), HOMEIMPROVEMENT (mejora del hogar) y DEBTCONSOLIDATION (consolidación de deudas).

In [ ]:
variable = data["loan_intent"]
variable.unique()

In [ ]:
pd.get_dummies(variable, prefix="loan_intent_")

#### Loan Int Rate (tipo de interés)

In [ ]:
variable = data["loan_int_rate"]
variable.hist(bins=31)

#### Loan Percent Income (pct del préstamo respecto a los ingresos anuales)

In [ ]:
variable = data["loan_percent_income"]
variable.hist(bins=31)

In [ ]:
variable = np.log(variable+1e-1)
variable.hist(bins=31)

#### Cb Person Cred Hist Length (duración del crédito)

In [ ]:
variable = data["cb_person_cred_hist_length"]
variable.hist(bins=31)

In [ ]:
variable = np.log(variable)
variable.hist(bins=31)

#### Credit Score

In [ ]:
variable = data["credit_score"]
variable.hist(bins=31)

#### Previous Loan Defaults on File (impagos anteriores)

In [ ]:
variable = data["previous_loan_defaults_on_file"]
variable.unique()

In [ ]:
variable.replace({"No": 0, "Yes": 1})

#### TODO JUNTO

In [ ]:
data["person_age"] = np.log(np.clip(data["person_age"], 0, 70))
data["person_gender"] = data["person_gender"].replace({"female":0, "male":1})
data["person_education"] = data["person_education"].replace({"High School":0, "Associate":1, "Bachelor":2, "Master":3, "Doctorate":4})
data["person_income"] = np.log(data["person_income"])
data["person_emp_exp"] = np.log(data["person_emp_exp"]+1)
data = pd.concat((data, pd.get_dummies(data["person_home_ownership"], prefix="person_home_ownership_")), axis=1)
data = data.drop(columns=["person_home_ownership"])
data["loan_amnt"] = np.log(data["loan_amnt"])
data = pd.concat((data, pd.get_dummies(data["loan_intent"], prefix="loan_intent_")), axis=1)
data = data.drop(columns=["loan_intent"])
data["loan_percent_income"] = np.log(data["loan_percent_income"]+0.1)
data["cb_person_cred_hist_length"] = np.log(data["cb_person_cred_hist_length"])
data["previous_loan_defaults_on_file"] = data["previous_loan_defaults_on_file"].replace({"No": 0, "Yes": 1})

In [ ]:
data.info()

### Separación en TRAIN-TEST y normalización

In [ ]:
X = data.drop(columns=["loan_status"])
y = data["loan_status"]

In [ ]:
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
means = X_train_raw.mean()
stds = X_train_raw.std()
X_train = (X_train_raw-means) / stds
X_test = (X_test_raw-means) / stds

In [ ]:
X_train.describe()

In [ ]:
columnas = X_train_raw.columns
columnas

In [ ]:
X_train = X_train.values
y_train = y_train.values
X_test = X_test.values
y_test = y_test.values

In [ ]:
X_train.shape, y_train.shape

## <font color="#F48E16">Modelos</font>

### Regresión Logística (modelo lineal)

In [ ]:
rl = LogisticRegression()
rl.fit(X_train, y_train)
pred = rl.predict(X_test)
rl.score(X_test, y_test)

In [ ]:
sns.heatmap(confusion_matrix(y_test, pred, normalize="true"), annot=True)
plt.ylabel("Real")
plt.xlabel("Predicción")
plt.show()

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel().tolist()

print(" > ACCURACY:", (tp+tn)/(tp+tn+fp+fn))
print(" > PRECISION:", tp/(tp+fp))
print(" > RECALL:", tp/(tp+fn))

### SVM

In [ ]:
svc = SVC(kernel="rbf", C=1.0, gamma="scale", max_iter=2000, probability=True)
svc.fit(X_train, y_train)
pred = svc.predict(X_test)
svc.score(X_test, y_test)

In [ ]:
sns.heatmap(confusion_matrix(y_test, pred, normalize="true"), annot=True)
plt.ylabel("Real")
plt.xlabel("Predicción")
plt.show()

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel().tolist()

print(" > ACCURACY:", (tp+tn)/(tp+tn+fp+fn))
print(" > PRECISION:", tp/(tp+fp))
print(" > RECALL:", tp/(tp+fn))

### MLP

In [ ]:
red = MLPClassifier(hidden_layer_sizes=(100,), max_iter=2000)
red.fit(X_train, y_train)
pred = red.predict(X_test)
red.score(X_test, y_test)

In [ ]:
sns.heatmap(confusion_matrix(y_test, pred, normalize="true"), annot=True)
plt.ylabel("Real")
plt.xlabel("Predicción")
plt.show()

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel().tolist()

print(" > ACCURACY:", (tp+tn)/(tp+tn+fp+fn))
print(" > PRECISION:", tp/(tp+fp))
print(" > RECALL:", tp/(tp+fn))

## <font color="#F48E16">Explicabilidad</font>

**IMPORTANTE**: Por simplicidad para los algoritmos más pesados, vamos a utilizar solo un trocito de X_train como si fuese nuestro dataset completo.

In [ ]:
X_train_small = X_train[:100]
y_train_small = y_train[:100]

### Importancia por permutación [Método global]

La importancia por permutación consiste en permutar uno a uno los atributos del dataset de entrada y evaluar el modelo M para ver cómo se modifica su rendimiento. La relevancia, por tanto, se define así:

$$R_i = M(X) - \frac{1}{N} \sum_{j=1}^N M(X'_i)$$

donde $R_i$ es la relevancia del atributo $i$-ésimo, $M(X)$ representa la métrica de rendimiento del modelo $M$ a partir del dataset original $X$, $N$ es el número de repeticiones para promediar, y $X'_i$ hace referencia al dataset original X habiendo permutado el atributo $i$-ésimo.

**¿Por qué se hace esto?**

Porque si se pierde rendimiento cuando se rompe el atributo $i$-ésimo, esto significa que dicho atributo es importante para el modelo.

Este algoritmo de explicabilidad es un método global, ya que utiliza todo el dataset para dar un valor global de la relevancia de un atributo.

In [ ]:
def importancia_permutacion(X, y, model, N=10):
    Rx = np.zeros(X.shape[1])

    ##################################
    # TO-DO Implementa el algoritmo de importancia por permutación
    ##################################

    return Rx

In [ ]:
Rx_perm_rl = importancia_permutacion(X_train_small, y_train_small, rl, N=10)
Rx_perm_svc = importancia_permutacion(X_train_small, y_train_small, svc, N=10)
Rx_perm_red = importancia_permutacion(X_train_small, y_train_small, red, N=10)

In [ ]:
plt.figure(figsize=(10, 3))
plt.bar(columnas, Rx_perm_rl)
plt.xticks(rotation=90)
plt.grid(alpha=0.2)
plt.show()

In [ ]:
plt.figure(figsize=(10, 3))
plt.bar(columnas, Rx_perm_svc)
plt.xticks(rotation=90)
plt.grid(alpha=0.2)
plt.show()

In [ ]:
plt.figure(figsize=(10, 3))
plt.bar(columnas, Rx_perm_red)
plt.xticks(rotation=90)
plt.grid(alpha=0.2)
plt.show()

In [ ]:
x = np.arange(len(columnas))
width = 0.25

plt.figure(figsize=(10, 3))
plt.bar(x - width, Rx_perm_rl, width, label="Regresión Logística")
plt.bar(x, Rx_perm_svc, width, label="SVC")
plt.bar(x + width, Rx_perm_red, width, label="Red Neuronal")
plt.xticks(x, columnas, rotation=90)
plt.ylabel("Importancia por Permutación")
plt.legend()
plt.grid(alpha=0.2)
plt.show()

**¿Conclusiones?**

Yo diría que...

- Si para 3 modelos distintos hay atributos que tienen siempre relevancia con magnitud bajita, es que esos atributos sobran en el dataset.

- Si hay algunos atributos que siempre tienen relevancia alta, ya sea por arriba o por abajo, es que son relevantes en el dataset.

- La clave está en que algunos atributos son útiles para algún modelo pero inútiles para otro. Eso significa que estás explicando el modelo en particular y no seleccionando atributos en general.

### Relevancia por oclusión [Método local y global]

La relevancia por oclusión, en general, se utiliza como técnica de explicabilidad de modelos que procesan imágenes, pero, aunque es una técnica que veremos más adelante aplicada a dicha tarea, también puede ser utilizada en problemas tabulares. La relevancia por oclusión es un método híbrido, que puede ser aplicado de forma tanto local (para una observación concreta) como global (para todo el dataset), que consiste en anular una región del espacio de atributos de entrada.

El término anular consiste en reemplazar un atributo $i$-ésimo por un valor "inteligente", por ejemplo:

- Un valor fijo 0

- La media del atributo

- La moda del atributo

- Un valor imputado mediante un algoritmo más complejo

La relevancia por oclusión global para datos tabulares se calcula, entonces, de la siguiente manera:

$$R_i = M(X) - M(X'_i)$$

donde $M(X)$ representa una métrica de rendimiento del modelo $M$ al procesar el dataset $X$ y $M(X'_i)$ representa la misma métrica de rendimiento al procesar el dataset con el atributo $i$-ésimo anulado.

Sin embargo, si se busca la oclusión a nivel local, se calcularía de la siguiente manera:

$$R_i(x) = f(x) - f(x'_i)$$

donde $f(x)$ es la predicción del modelo para el dato de entrada $x$ y $f(x'_i)$ es la predicción del mismo modelo para el dato de entrada $x$ con el atributo $i$-ésimo anulado.

**¿Por qué se sigue esta estrategia?**

Esta estrategia se basa en una idea parecida a la importancia por permutación, donde se evalúa el modelo después de modificar un atributo de entrada del dataset.

En el siguiente código se va a utilizar el valor fijo 0 porque es la media de los atributos de entrada, ya que están normalizados.

In [ ]:
def relevancia_oclusion(X, y, model, verbose=True):
  if X.ndim == 1: # 1 solo dato, lo ponemos en formato matricial
    X = X[None, :]
    pred = model.predict_proba(X)
  else:
    score = model.score(X, y)

  num_atributos = X.shape[-1]

  Rx = np.zeros(num_atributos)
  ##################################
  # TO-DO Implementa la relevancia por oclusión tanto global como local
  ##################################

  return Rx

**Relevancia global**

In [ ]:
Rx_oclu_rl = relevancia_oclusion(X_train_small, y_train_small, rl)
Rx_oclu_svc = relevancia_oclusion(X_train_small, y_train_small, svc)
Rx_oclu_red = relevancia_oclusion(X_train_small, y_train_small, red)

In [ ]:
x = np.arange(len(columnas))
width = 0.3

plt.figure(figsize=(10, 3))
plt.bar(x - width/2, Rx_perm_rl, width, label="Importancia por permutación")
plt.bar(x + width/2, Rx_oclu_rl, width, label="Relevancia por oclusión")
plt.xticks(x, columnas, rotation=90)
plt.title("Regresión Logística")
plt.legend()
plt.grid(alpha=0.2)
plt.show()

In [ ]:
x = np.arange(len(columnas))
width = 0.3

plt.figure(figsize=(10, 3))
plt.bar(x - width/2, Rx_perm_svc, width, label="Importancia por permutación")
plt.bar(x + width/2, Rx_oclu_svc, width, label="Relevancia por oclusión")
plt.xticks(x, columnas, rotation=90)
plt.title("SVC")
plt.legend()
plt.grid(alpha=0.2)
plt.show()

In [ ]:
x = np.arange(len(columnas))
width = 0.3

plt.figure(figsize=(10, 3))
plt.bar(x - width/2, Rx_perm_red, width, label="Importancia por permutación")
plt.bar(x + width/2, Rx_oclu_red, width, label="Relevancia por oclusión")
plt.xticks(x, columnas, rotation=90)
plt.title("Red Neuronal")
plt.legend()
plt.grid(alpha=0.2)
plt.show()

Como hemos podido ver, **dos métodos diferentes de explicabilidad dan dos resultados también diferentes**.

**Relevancia local**

Ahora vamos a explicar un dato particular.

In [ ]:
item = 0

In [ ]:
Rx_oclu_local_rl = relevancia_oclusion(X_train_small[item], y_train_small[item], rl)
Rx_oclu_local_svc = relevancia_oclusion(X_train_small[item], y_train_small[item], svc)
Rx_oclu_local_red = relevancia_oclusion(X_train_small[item], y_train_small[item], red)

In [ ]:
x = np.arange(len(columnas))
width = 0.3

plt.figure(figsize=(10, 3))
plt.bar(x - width/2, Rx_oclu_rl, width, label="Global")
plt.bar(x + width/2, Rx_oclu_local_rl, width, label="Local item="+str(item))
plt.xticks(x, columnas, rotation=90)
plt.title("Relevancia por oclusión con Regresión Logística")
plt.legend()
plt.grid(alpha=0.2)
plt.show()

¿Por qué con el item=0 pasa que loan_percent_income, por ejemplo, es negativo?

Primero vemos el dato original y su predicción:

In [ ]:
plt.figure(figsize=(10, 3))
plt.subplot(1, 2, 1)
plt.bar(columnas, X_train_small[item])
plt.xticks(rotation=90)
plt.grid(alpha=0.2)
plt.subplot(1, 2, 2)
plt.plot([0, 1], rl.predict_proba(X_train_small[item][None, :])[0], "o")
plt.xticks([0, 1], ["Clase 0", "Clase 1"])
plt.title("Clase real: " + str(y_train_small[item]))
plt.grid()
plt.show()

Vamos a anular **loan_percent_income**, es decir, hacerlo más pequeño.

In [ ]:
X_null = X_train_small[item].copy()
X_null = X_null[None, :]
X_null[:, 7] = 0

plt.figure(figsize=(10, 3))
plt.subplot(1, 2, 1)
plt.bar(columnas, X_null[0])
plt.bar(columnas, X_train_small[item], alpha=0.2)
plt.xticks(rotation=90)
plt.grid(alpha=0.2)
plt.subplot(1, 2, 2)
plt.plot([0, 1], rl.predict_proba(X_train_small[item][None, :])[0], "o", label="Original")
plt.plot([0, 1], rl.predict_proba(X_null)[0], 'o', label="Anulado")
plt.xticks([0, 1], ["Clase 0", "Clase 1"])
plt.grid()
plt.legend()
plt.show()

# ¿Tiene sentido?
# Si anulas un atributo cuya explicabilidad con oclusión es negativa,
# la predicción de la clase real AUMENTA

Vamos a anular ahora **loan_amnt**, es decir, bajarlo.

In [ ]:
X_null = X_train_small[item].copy()
X_null = X_null[None, :]
X_null[:, 5] = 0

plt.figure(figsize=(10, 3))
plt.subplot(1, 2, 1)
plt.bar(columnas, X_null[0])
plt.bar(columnas, X_train_small[item], alpha=0.2)
plt.xticks(rotation=90)
plt.grid(alpha=0.2)
plt.subplot(1, 2, 2)
plt.plot([0, 1], rl.predict_proba(X_train_small[item][None, :])[0], "o", label="Original")
plt.plot([0, 1], rl.predict_proba(X_null)[0], 'o', label="Anulado")
plt.xticks([0, 1], ["Clase 0", "Clase 1"])
plt.grid()
plt.legend()
plt.show()

# ¿Tiene sentido?
# Si anulas un atributo cuya explicabilidad con oclusión es positiva,
# la predicción de la clase real DISMINUYE

### SHAP (SHapley Additive exPlanations)

El valor de Shapley en teoría de juegos para una característica $i$-ésima es el siguiente:

$$\phi_i(x) = \sum_{S\subseteq F\setminus \{i\}} \frac{|S|!(|F|-|S|-1)!}{|F|!} [f_{S\cup \{i\}}(x) - f_S(x)]$$

donde $F$ es el conjunto de todos los atributos (features) y $f_S(x)$ es la predicción del modelo cuando sólo usamos los atributos de $S$, imputando el resto. Es decir, hay que calcular todas las permutaciones posibles de todos los atributos con todos los atributos excepto el $i$-ésimo. Como esto es exponencial y no es viable, lo que se hace en la práctica es **muestrear permutaciones aleatorias de atributos**.

¿En qué consiste la idea? En ir introduciendo de forma aleatoria atributos a la instancia del dataset, imputando el resto de atributos (con la media por ejemplo) y evaluando cómo afecta incorporar ese atributo a la decisión del modelo. Podemos basarnos en esta idea:

$$contribución_i = f_{S\cup \{i\}}(x) - f_S(x)$$

SHAP es, por tanto, un método de explicabilidad local que utiliza la imputación conforme al total de los datos para explicar una instancia particular del dataset.

In [ ]:
def shap(X, y, model, item, N=10, verbose=True):
    num_atributos = X.shape[1]
    shap_values = np.zeros(num_atributos)
    x = X[item][None, :]
    y = y[item]

    ##################################
    # TO-DO Implementa SHAP
    ##################################

    return shap_values / N

In [ ]:
Rx_shap_rl = shap(X_train_small, y_train_small, rl, item=0, N=50)

In [ ]:
x = np.arange(len(columnas))
width = 0.3

plt.figure(figsize=(10, 3))
plt.bar(x - width/2, Rx_shap_rl, width, label="SHAP")
plt.bar(x + width/2, Rx_oclu_local_rl, width, label="Oclusión local")
plt.xticks(x, columnas, rotation=90)
plt.title("Relevancia por oclusión con Regresión Logística")
plt.legend()
plt.grid(alpha=0.2)
plt.show()

**Observación**: Relevancia por oclusión y SHAP dan una relevancia que se basa en la misma idea. Añadir o quitar atributos y ver la variación en la respuesta del modelo. Tiene sentido que sean resultados parecidos.

### LIME (Local Interpretable Model-agnostic Explanations)

LIME es un algoritmo de explicabilidad local basado en la idea de aproximar el comportamiento del modelo complejo con puntos cercanos a la instancia que se quiere explicar utilizando un modelo lineal.

¿Cómo funciona la idea?

1. Generamos un dataset sintético $X'$ alrededor de la instancia $x$.
2. Se calculan las predicciones del modelo complejo sobre $X'$.
3. Se asignan pesos a los puntos de $X'$ según su cercanía a la instancia $x$.
4. Se entrena un modelo lineal con los puntos de $X'$ según la cercanía.

Así, el modelo lineal aprende cómo cambian las predicciones alrededor de $x$. Además, **los pesos del modelo lineal son los valores de relevancia de cada atributo**.

LIME, al contrario que SHAP, no hace combinaciones exhaustivas, sino que solo mira el vecindario cercano a la instancia $x$.

In [ ]:
def lime(X, y, model, item, D=1000):
  num_atributos = X.shape[1]
  x0 = X[item][None, :]
  y = y[item]

  Rx = None

  ##################################
  # TO-DO Implementa LIME
  ##################################

  return Rx

In [ ]:
Rx_lime_rl = lime(X_train_small, y_train_small, rl, 0)

In [ ]:
x = np.arange(len(columnas))
width = 0.25

plt.figure(figsize=(10, 3))
plt.bar(x - width, Rx_shap_rl, width, label="SHAP")
plt.bar(x, Rx_oclu_local_rl, width, label="Oclusión local")
plt.bar(x + width, Rx_lime_rl, width, label="LIME")
plt.xticks(x, columnas, rotation=90)
plt.title("Relevancia con Regresión Logística")
plt.legend()
plt.grid(alpha=0.2)
plt.show()

# WOW ¿Qué está pasando en LIME? previous_loan_defaults_on_file tiene el signo cambiado?

Vamos a volver a ver el dato original

In [ ]:
plt.figure(figsize=(10, 3))
plt.subplot(1, 2, 1)
plt.bar(columnas, X_train_small[item])
plt.xticks(rotation=90)
plt.grid(alpha=0.2)
plt.subplot(1, 2, 2)
plt.plot([0, 1], rl.predict_proba(X_train_small[item][None, :])[0], "o")
plt.xticks([0, 1], ["Clase 0", "Clase 1"])
plt.title("Clase real: " + str(y_train_small[item]))
plt.grid()
plt.show()

Fijaos la diferencia entre SHAP y Oclusión frente a LIME. ¿Cómo interpretamos las cosas? Os dejo por aquí una tabla resumen teniendo en cuenta que:

- SHAP y Oclusión **anulan atributos para definir la relevancia**
- LIME **analiza la dirección del cambio**

Por tanto:

<table>
  <tr>
    <td><b>Método</b></td>
    <td><b>Interpretación</b></td>
  </tr>
  <tr>
    <td><b>SHAP y Oclusión</b></td>
    <td>Ri > 0 --> el atributo aumenta la predicción respecto al baseline, por lo que <b>anular el atributo disminuye la predicción</b>.<br>
    Ri < 0 --> el atributo disminuye la predicción respecto al baseline, por lo que <b>anular el atributo aumenta la predicción</b>.</td>
  </tr>
  <tr>
    <td>LIME</td>
    <td>Ri > 0 --> <b>aumentar el atributo aumenta la predicción</b><br>
    Ri < 0 --> <b>aumentar el atributo disminuye la predicción</b></td>
  </tr>
</table>

**Observación IMPORTANTE**: Como los datos están normalizados, anular **no siempre significa hacer más pequeño**, sino hacerlo 0. Si yo tengo $x_i = -2$, anular es hacer $x_i = 0$ (estoy aumentando el valor del atributo).

**Observación**: ¿Puedo saber la dirección de cambio con SHAP o Oclusión? Sí, simplemente multiplicando por el signo del atributo que estás evaluando: $R_i = R_i \times sign(x_i)$. Hagámoslo con los atributos **loan_amnt** y **previous_loan_defaults_on_file**:

In [ ]:
# Loan ammount
print(" > LOAN_AMNT")
print("   valor real:", X_train_small[item][5])
print("   SHAP:", Rx_shap_rl[5])
print("     que significa que si voy hacia baseline (0.0), la predicción de la clase real baja porque estoy bajando")
print("   SHAP x sign(input):", Rx_shap_rl[5] * np.sign(X_train_small[item][5]))
print("     que significa que si voy en sentido positivo, la predicción de la clase real sube")
print("   LIME:", Rx_lime_rl[5])
print("     que significa que si voy en sentido positivo, la predicción de la clase real sube")
print()

# previous_loan_defaults_on_file
print(" > previous_loan_defaults_on_file")
print("   valor real:", X_train_small[item][10])
print("   SHAP:", Rx_shap_rl[10])
print("     que significa que si voy hacia baseline (0.0), la predicción de la clase real sube porque estoy subiendo")
print("   SHAP x sign(input):", Rx_shap_rl[10] * np.sign(X_train_small[item][10]))
print("     que significa que si voy en sentido positivo, la predicción de la clase real sube")
print("   LIME:", Rx_lime_rl[10])
print("     que significa que si voy en sentido positivo, la predicción de la clase real sube")

**¿Conclusiones? ¿Dudas?**

- El signo de **SHAP te indica lo contrario de lo que va a pasar si anulas el atributo**. Si SHAP es negativo y anulas, la predicción sube. Si SHAP es positivo y anulas, la predicción baja.

- **LIME te indica la dirección para aumentar la clase real**. Si LIME es positivo, si aumentas el atributo, la predicción sube. Si LIME es negativo, si aumentas el atributo, la predicción baja.

- Multiplicar por el signo de la instancia (`np.sign(x)`) alterna entre las dos opciones, es decir, **SHAP·signo indica la dirección para aumentar la clase real**, igual que LIME; y **LIME·signo indica lo contrario de lo que va a pasar si anulas el atributo**, igual que SHAP.

- ¿Qué método es mejor?